## AI - based Sonnet Generation

Note: Go to the last cells for examples of outputs

To try for yourself:

1.   Download the repo and 
2.   Create model using the ``` create_model() ``` function below
3.   Load model weights ``` model.load_weights('<folder_path>/model_2_wts.h5' ```
4.   Run the last cell to generate new sonnets



In [ ]:
!pip install pyyaml h5py
import os

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/ieee'
%ls

Mounted at /gdrive
/gdrive/My Drive/ieee
'=1.16.6'                 saved_model2/
 assets/                  saved_model.pb
 baseline_weights_01.h5  'Sonnet generation example 1.mp4'
 dataset2.txt             sonnet_generation_service.py
 dataset.txt              tokenizer.pickle
 iambic.txt               Untitled.ipynb
 model2/                  variables/
 model_2_wts.h5           video.mp4
 my_model/                weights.h5
 pickled_model.pkl        write_sonnet_01.py
 __pycache__/


In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [14]:
# Suppress Tensorflow warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
# Create Tokenizer object for text preprocessing
tokenizer = Tokenizer()

# Read data
data = open('dataset2.txt').read()
corpus = data.lower().split("<eos>")
#print(corpus)

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
#print(tokenizer.word_index)
#print(total_words)

In [5]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'in': 5,
 'a': 6,
 'that': 7,
 'i': 8,
 'with': 9,
 'my': 10,
 "'s": 11,
 'his': 12,
 'all': 13,
 'for': 14,
 'but': 15,
 'from': 16,
 'her': 17,
 'thy': 18,
 'is': 19,
 'as': 20,
 'on': 21,
 "'d": 22,
 'he': 23,
 'me': 24,
 'not': 25,
 'their': 26,
 'so': 27,
 'this': 28,
 'by': 29,
 'love': 30,
 'thou': 31,
 'or': 32,
 'be': 33,
 'it': 34,
 '—': 35,
 'yet': 36,
 'when': 37,
 'they': 38,
 'we': 39,
 'no': 40,
 'like': 41,
 '’': 42,
 'which': 43,
 'our': 44,
 'at': 45,
 'you': 46,
 'are': 47,
 'was': 48,
 'if': 49,
 'she': 50,
 'then': 51,
 'more': 52,
 '“': 53,
 'what': 54,
 'thee': 55,
 'who': 56,
 'have': 57,
 'now': 58,
 'heart': 59,
 'nor': 60,
 'one': 61,
 '‘': 62,
 'some': 63,
 'where': 64,
 'your': 65,
 'still': 66,
 'shall': 67,
 'may': 68,
 'life': 69,
 'will': 70,
 'day': 71,
 'there': 72,
 'them': 73,
 'eyes': 74,
 'than': 75,
 'did': 76,
 'him': 77,
 'o': 78,
 'its': 79,
 'soul': 80,
 'can': 81,
 'world': 82,
 'these': 83,
 'had':

In [6]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
max_sequence_len

15

In [ ]:
import pickle

# Save tokenizer object in pickle file
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(xs[6])

[   0    0    0    0    0    0    0   96  105  650   54 1826   34    7]


In [ ]:
print(ys[6])

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
print(xs[5])
print(ys[5])

[   0    0    0    0    0    0    0    0   96  105  650   54 1826   34]
[0. 0. 0. ... 0. 0. 0.]


###Model creation and training

In [7]:
def create_model():
  model = Sequential()
  model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
  model.add(Bidirectional(LSTM(150)))
  model.add(Dense(total_words, activation='softmax'))
  adam = Adam(lr=0.01)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

  return model

In [ ]:
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
#plot_graphs(history, 'accuracy')
# estimating 7 words per line => 7*13 = 91

In [ ]:
model.save('/gdrive/My Drive/ieee')
model.save_weights('/gdrive/My Drive/ieee/model_2_wts.h5')

In [ ]:
# Save trained model as pickle file

import joblib
import tempfile

from tensorflow.keras.models import Model, load_model, save_model

# Hotfix function
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

# Run the function
make_keras_picklable()

# Save
with open("/gdrive/My Drive/ieee/pickled_model.pkl", 'wb') as f:
    joblib.dump(model_to_save, f)


In [10]:
# Load saved model
model_to_save = create_model()
model_to_save.load_weights('/gdrive/My Drive/ieee/model_2_wts.h5') 
# model_to_save.load_weights('<path_to_repository>/model_2_wts.h5') # Replace folder path as appropriate

## **Enter seed text to generate a new sonnet!**

In [25]:
# Get prefix from user
seed_text = input("Enter seed text/prefix: ")

# Print sonnet
next_words = 91
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model_to_save.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

seed_text = seed_text.split(" ")

print("\nHello dummy Shakespeare! Here's your sonnet: ")
i=0
for word in seed_text: 
    if i%7 == 0:
        print()
    if i%14 == 0:
      print()
    if i%28==0:
        print()
    print(word, end = " ") 
    i = i+1


Enter seed text/prefix: Forlorn I am 

Hello dummy Shakespeare! Here's your sonnet: 



Forlorn I am  sick of self 
and e'en and turns she made every 

want of fear of theirs might fear 
view none more il ’ s world 


we of of fire to kindred the 
lordly deck measure unless vines and of 

garin please of fear is garin branches 
of other place of of 4 god 


of of of me nor my elder 
sons bright garin void of sighs away 

known of isle her stone to dust 
fil there sighs doth host lit qu'on 


scorn highest his heavy fortune before away 
world venerable pleached twain 

In [ ]:
# Example 1
'''
The sky is bright and blue infinity 
spread and idaean jove in his line 

sees disguise a son above size of 
rain rest whose voice was divine my 


wind at half friendship what 's save 
my weeping feel me ’ t of 

wet man 's desire be hollow soul 
is just wit or faithful ride of 


blood world joys bowre dare at will 
fling black publick tell horses garin yon 

soul è d of six had whose 
eyes on us fear in me less 


a thunder de yon lower good power 
the soul weeps of mine chants 
'''

In [ ]:
# Example 2
'''
Enter seed text/prefix: Hello darkness, my old friend

Hello dummy Shakespeare! Here's your sonnet: 

Hello darkness, my old friend i trust 
no more bold and yet i have 

i spiritual light their way off and 
bade whose hair place and wind of 


fear of mine own those where time 
he fears to hand morn said of 

castle radiance too if i was but 
forward again ’ force to was put 


young so great of strength and yet 
their bold restored 'd if faithful de 

aquino good to say woof burn chased 
when his hand of women their bloody 


thrusting to fear me it did great 
bright raise their tide garin 
'''